In [ ]:
import requests
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch


In [ ]:
#-----Configuración de Activo----

#Open Interest limit max 500 , en las velas period es interval
symbol= 'BTCUSDT'
limit= '500'
period= '1h'

In [ ]:
#----Descarga de Datos de Taker Buy Sell Vol Binance-----
url = "https://dapi.binance.com/futures/data/takerBuySellVol"
params = {
    "pair": "BTCUSD",
    "contractType": "ALL",
    "period": period,
    "limit": limit
}

response = requests.get(url, params=params)
js = response.json()
data_tbsv = pd.DataFrame(js)

data_tbsv = data_tbsv.rename(columns={"timestamp":"fecha"})
data_tbsv.drop(['pair','contractType'], axis=1, inplace=True)
data_tbsv.set_index('fecha', inplace=True)
data_tbsv.index= pd.to_datetime(data_tbsv.index, unit='ms')

#Convertir los valores string a numero
data_tbsv= data_tbsv.apply(pd.to_numeric)
data_tbsv.columns

In [ ]:
#----Descarga datos del Precio de Binance----
url="https://fapi.binance.com/fapi/v1/klines"
params = {
    'symbol': symbol,
    'interval': period,
    'limit': limit
}
response = requests.get(url, params= params)
js = response.json()

#----Organizardatos de Klines Binance---
data = pd.DataFrame(js)
data = data.rename(columns={0:"fecha", 1:"Open", 2:"High", 3:"Low", 4:"Close", 5:"Volume"})
data.drop([6,7,8,9,10,11], axis=1, inplace=True)
data.set_index('fecha', inplace=True)
data.index= pd.to_datetime(data.index, unit='ms')

#Convertir los valores string a numero
data= data.apply(pd.to_numeric)


In [ ]:
data_tbsv['diff'] = data_tbsv['takerBuyVol'] - data_tbsv['takerSellVol']

In [ ]:
data = data.join(data_tbsv)

In [ ]:
data.drop(['takerBuyVolValue','takerSellVol','takerSellVolValue','takerBuyVol'], axis=1, inplace=True)

In [ ]:
# Utilizando numpy.where() para aplicar una condición y asignar valores
data['color'] = np.where(data['diff'] > 0, 'green', 'red')
data['diff'] = np.where(data['diff'] < 0, data['diff'] * -1, data['diff'])


In [ ]:
#Ultimos 150 datos del DataFrame
data = data.iloc[350:]

In [ ]:
# Crear la figura y los ejes del gráfico
fig, ax = plt.subplots(figsize=(20, 10))

# Crear las barras utilizando los valores de la columna
barras = ax.bar(data.index, data['diff'], width=0.01, ec=data['color'])

# Crear un segundo eje y
ax2 = ax.twinx()

# Plotear el precio en el segundo eje y
ax2.plot(data.index, data['Close'], color='blue')
ax2.set_ylabel('Precio de cierre')

# Personalizar el gráfico
ax.set_xlabel('Fecha')
ax.set_ylabel('Taker Buy/Sell Volume')
ax.set_title('API Binance Taker Buy/Sell Volume + Precio')

legend_elements = [
    Patch(facecolor='red', label='Taker Sell'),
    Patch(facecolor='green', label='Taker Buy'),
    Patch(facecolor='blue', label='Precio')
]

# Mostrar el gráfico de barras
plt.legend(handles=legend_elements, loc='upper right')
plt.show()    